In [77]:
from bs4 import BeautifulSoup             
import os
import re
from nltk.corpus import stopwords
import pandas as pd


import gensim
from gensim import models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

from gensim.corpora import Dictionary

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [78]:
stemmer = SnowballStemmer('english')
stops = set(stopwords.words("english"))

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def lyric_to_words( raw_lyric ):
    lyric_text = BeautifulSoup(raw_lyric, features="lxml").get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", lyric_text) 
    words = letters_only.lower().split()                                               
    meaningful_words = [w for w in words if not w in stops]
    lemmatize_stemming_words = [lemmatize_stemming(w) for w in words] 

    return lemmatize_stemming_words
    # return( " ".join( meaningful_words ))

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [79]:
data_set = pd.read_csv("tcc_ceds_music.csv", delimiter=",")


/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
%%time
processed_lyrics = data_set['lyrics'].map(lyric_to_words)

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
CPU times: user 25.6 s, sys: 138 ms, total: 25.7 s
Wall time: 25.6 s


In [81]:
processed_lyrics.head()

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    [hold, time, feel, break, feel, untru, convinc...
1    [believ, drop, rain, fall, grow, believ, darke...
2    [sweetheart, send, letter, goodby, secret, fee...
3    [kiss, lip, want, stroll, charm, mambo, chacha...
4    [till, darl, till, matter, know, till, dream, ...
Name: lyrics, dtype: object

In [82]:
%%time
dictionary = Dictionary(processed_lyrics)

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
CPU times: user 999 ms, sys: 0 ns, total: 999 ms
Wall time: 998 ms


In [83]:
%%time
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

CPU times: user 39.3 ms, sys: 0 ns, total: 39.3 ms
Wall time: 38.8 ms
/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [84]:
%%time
corpus = [dictionary.doc2bow(lyric) for lyric in processed_lyrics]

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
CPU times: user 564 ms, sys: 7.69 ms, total: 572 ms
Wall time: 571 ms


In [85]:
%%time
lda_model = gensim.models.LdaMulticore(corpus, num_topics=5, id2word=dictionary, passes=2)

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
CPU times: user 5.25 s, sys: 493 ms, total: 5.74 s
Wall time: 5.85 s


In [86]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.020*"know" + 0.015*"time" + 0.014*"go" + 0.013*"heart" + 0.012*"like" + 0.012*"leav" + 0.011*"come" + 0.010*"world" + 0.010*"fall" + 0.009*"mind"
Topic: 1 
Words: 0.021*"heart" + 0.017*"away" + 0.016*"know" + 0.015*"like" + 0.013*"life" + 0.013*"come" + 0.010*"want" + 0.009*"babi" + 0.008*"feel" + 0.008*"littl"
Topic: 2 
Words: 0.020*"come" + 0.016*"like" + 0.013*"yeah" + 0.012*"know" + 0.009*"gonna" + 0.009*"want" + 0.008*"time" + 0.008*"hear" + 0.007*"song" + 0.007*"babi"
Topic: 3 
Words: 0.023*"time" + 0.020*"like" + 0.020*"know" + 0.014*"feel" + 0.012*"live" + 0.012*"right" + 0.010*"come" + 0.009*"away" + 0.009*"night" + 0.009*"want"
Topic: 4 
Words: 0.015*"know" + 0.011*"like" + 0.010*"go" + 0.009*"come" + 0.009*"babi" + 0.009*"heart" + 0.009*"tear" + 0.008*"walk" + 0.008*"caus" + 0.007*"stand"
/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the fut

In [87]:
for index, score in sorted(lda_model[corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.990237832069397	 
Topic: 0.023*"time" + 0.020*"like" + 0.020*"know" + 0.014*"feel" + 0.012*"live" + 0.012*"right" + 0.010*"come" + 0.009*"away" + 0.009*"night" + 0.009*"want"
/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [88]:
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(
    lda_model,
    corpus,
    dictionary,
    sort_topics=False,
)
pyLDAvis.display(lda_display)

/home/gabriel/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
